In [1]:
import tensorflow as tf
import keras
import random
import numpy as np
import cv2
from collections import deque
import time

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
import gym

In [5]:
#parameters
ENV_NAME='Breakout-v0'
#今回は画像データで深層強化学習を進める
FRAME_WIDTH=84
FRAME_HEIGHT=84

#学習回数
MAX_EPISODES=12000
STATE_LENGTH=4#networkに入れるフレーム数
GAMMA=0.99#割引率
EXPLORATION_STEPS=10000#一回学習当たりのステップ数

INITIAL_EPSILON=1.0#ε-greedyの最初のε
FINAL_RPSION=0.1

INITIAL_REPLAY_SIZE=2000#学習を始める前に記憶メモリから取ってくるメモリの容量
NUM_REPLAT_MEMORY=20000#メモリの容量
BATCH_SIZE=32

TARGET_UPDATE_INTERVAL=20#TARGET_NETWORKを更新する間隔

ACTION_INTERVAL=4
TRAIN_INTERVAL=4#agentはupdateするときに四つの行動をとる

LEARNING_RATE=0.0001#RMSpropの学習率
MOMENTUM=0.95
MIN_GRAD=0.01

SAVE_INTERVAL=3000#networkを保存する回数
NO_OP_STEPS=30#agentが何もしないという行動の最大数

LOAD_NETWORK=False#学習したモデルをロードするかしないか

TRAIN=True

SAVE_NETWORK_PATH='saved_network/'+ENV_NAME



In [8]:
class Agent():
    
    def __init__(self,num_actions):
        self.num_actions=num_actions
        self.epsilon=INITIAL_EPSILON#行動選択
        self.epsilon_step=(FINAL_EPSILON-INITIAL_EPSILON)/EXPLORATION_STEPS
        self.t=0
        self.repeated_action=0#一つ前の行動のリスト
        
        #学習のために必要なパラメタ
        self.total_reward=0#報酬合計和
        self.total_q_max=0#最大価値関数の合計
        self.total_loss=0#Q関数の合計損失
        self,episode=0#学習回数(now)
        self.duratuin=0#?
        
        #学習時間を計測するため　初期値
        self.start=0
        
        #memory
        self.replay_memory=deque()
        
        #q_networkの構築
        
        #main
        self.s,self.main_q_values,main_q_network=self.net()
        main_q_network_weights=main_q_network.trainable_weights#main_q_networkの初期パラメタ
        
        #target
        self.st,self.target_q_values,target_network=self.net()
        target_q_network_weights=target_q_network.trainable_weights
        
        #main_q_networkのパラメータで更新するためのリスト
        self.update_target_network=[target_network_weights[i].assign(main_q_network_weights[i]) for i in range(len(target_q_network_weights))]
        
        #最適化アルゴリズム指標
        """a:actionのvector,y:教師信号,loss,optimizer"""
        self.a,self.y,self.loss,self.optimizer=self.train_op(main_q_network_weights)
        
        
        #tensorflowで実行
        self.sess=tf.InteractiveSession()#with文がいらない
        self.saver=tf.train.Saver(main_q_network_weights)#保存
        
        #listの作成
        if not os.path.exists(SAVE_NETWORK_PATH):
            os.mkdir(SAVE_NETWORK_PATH)
            
        #学習パラメータの初期化
        self.sess.run(tf.global_variables_initializer())
        
        #load network すでに学習したモデルを用いて環境実行を行う
        if LOAD_NETWORK:
            self.load_network()
            
            
        #target_q_network_weightsを初期化
        self.sess.run(self.update_target_network)
        
        
        
        
    
        
        
    def net(self):
        """入力データ、q値、深層ネットワークモデルを返す"""
        model=Sequential()
        model.add(Conv2D(32,8,strides=(4,4),activation='relu',input_shape=(STATE_LENGTH,FRAME_WIDTH,FRAME_HEIGHT)))
        model.add(Conv2D(64,4,strides=(2,2),activation='relu'))
        model.add(Conv2D(64,3,strides=(1,1),activation='relu'))
        model.add(Flatten())
        model.add(Dense(512,activation='relu'))
        model.add(Dense(self,num_actions))
        
        #入力する画像データ
        img=tf.placeholder(dtype='float32',shape=[None,STATE_LENGTH,FRAME_WIDTH,FRAME_HEIGHT])
        #データをモデルに入れて出てくるのがQ値
        q_values=model(img)
        
        return img,q_values,model
    
    
    def train_op(self,q_network_weights):
        a=tf.placeholder(tf.int64,[None])#行動
        y=tf.placeholder(tf.float32,[None])#教師信号
        
        #onehot vector化
        a_onehot=tf.one_hot(a,self.num_actions,1.0,0.0)#shape=(batch_size,num_actions)
        q_value=tf.reduce_sum(tf.multiply(self.main_q_values,a_onehot),reduction_indices=1)#shape=(batch_size,)
        
        #正則化も含めて
        error=tf.abs(q_value-y)
        quardratic=tf.clip_by_value(error,0.0,1.0)#errorがこの値に収束するように設定するためのclip_by_value
        loss=tf.reduce_mean(0.5*tf.square(quardratic)+(error-quardratic))
        
        optimizer=tf.train.RMSPropOptimizer(LEARNING_RATE,momentum=MOMENTUM,epsilon=MIN_GRAD)
        optimizer=optimizer.minimize(loss,var_list=q_network_weights)#更新するパラメータq_network_weights
        
        return a,y,loss,optimizer
    
    
    def get_action(self,state):
        action=self.repeated_action
        
        if self.t%ACTION_INTERVAL==0:
            if self.epsilon>=np.random.rand() or self.t<ACTION_INTERVAL:
                action=random.randrange(self.num_actions)#行動の中からランダムに選ぶ
            else:
                #推論モードのq_valueで最大のものを選ぶQ(s(t+1),a)
                
                #stateの入力データが必要
                action=np.argmax(self.main_q_values.eval(feed_dict={self.s:[np.float32(state/255)]}))
                
            self.repeated_action=action
            
        #epsilonの更新
        if self.epsilon>FINAL_EPSILON and self.t>=INITIALIZE_REPLAY_SIZE:
            self.epsilon-=self.epsilon_step
            
            
        return action
    
    #観測(observation)を状態(state)に変える
    def get_initial_state(self,observation,before_observation):
        preprocessed_observation=np.maximum(observation,before_observation)
        #カラー画像をグレースケールに
        preprocessed_observation=cv2,cvtColor(preprocessed_observation,cv2.COLOR_RGB2GRAY)
        #resize
        preprocessed_observation=cv2.resize(preprocessed_observation,(FRANE_WIDTH,FRAME_HEIGHT))
        
        preprocessed_observation=np.uint8(preprocessed_observation*255)
        
        state=[preprocessed_observation for _ in range(STATE_LENGTH)]#一度に入れる状態の数のリストを作成
        
        state=np.stack(state,axis=0)#行方向に結合
        return state
    
    
    def run(self,state,action,reward,observation_next,terminal):
        """次の状態を返すための操作を行う関数
        　 環境(env)の実行(step(action))によって生成されるデータは、observation_next,reward,terminal,(info)
           observation_nextは次の状態、rewardは報酬、terminalは学習終了判定
           env runによりobservation_next,reward,terminalを取得"""
        next_state=np.append(state[1:,:,:],observation_next,axis=0)#行方向に足していく
        
        #報酬の渡し方 符号によって決定
        reward=np.sign(reward)
        
        #memoryに保存 transitionsともいう
        self.replay_memory.append((state,action,reward,next_state,terminal))
        
        #memoryの容量をオーバーしたとき
        if len(self.replay_memory)> NUM_REPLAY_MEMORY:
            self.replay_memory.popleft()#最初の方のデータは捨てる
         
        #Q関数をupdate
        if self.t>= INITIALIZE_REPLAY_SIZE:
            
            #update_q_network
            if self.t%TRAIN_INTERVAL==0:
                self.replay()
            
            #update_target_network
            if self.t%TARGET_UPDATE_INTERVAL==0:
                self.sess.run(self.update_target_network)
                
            if self.t%SAVE_INTERVAL==0:
                save_path=self.saver.save(self.sess,SAVE_NETWORK_PATH+'/'+ENV_NAME,global_step=self.t)
                print('success saved')
                
        self.total_reward+=reward
        self.total_q_max=np.max(self.main_q_values.eval(feed_dict={self.s:[np.float32(state/255)]}))
        self.duration+=1
        
        
        #終了した場合の処理
        if terminal:
            
            elapsed=time.time()-self.start
            if self.t<INITIALIZE_REPLAY_SIZE:
                mode='random'
            elif INITIALIZE_REPLAY_SIZE<=self.t<INITIALIZE_REPLAY_SIZE+EXPLORATION_STEPS:
                mode='exploration'
            else:
                mode='exploit'
            #終了したら合計値は初期化
            print('episode:{0:5d}/timestep:{1:8d}/duration{2:5d}/epsilon{3:3f}/total_reward{4:3.0f}/avg_max_q{5:2.4f}/avg_loss{6:.5f}/mode{7}/step_per_second{8:.1f}').format((
            self.episode+1,self.t,self.duration,self.epsilon,self.total_reward,self.total_q_max/float(self.duration),self.total_loss/(float(self.duration)/float(self.TRAIN_INTERVAL)),mode,self.duration/elapsed))
            self.total_reward=0
            self.total_q_max=0
            self.total_loss=0
            self.duration=0
            self.episode+=1
        self.t+=1
        
        return next_state
                
                
                
    def replay(self):
        #メモリから学習データを作成　バッチ学習
        state_batch=[]#状態s(t)のbatch
        action_batch=[]#行動a(t)のbatch
        reward_batch=[]#reward batch
        next_state_batch=[]#s(t+1)のbatch
        terminal_batch=[]#終了判定のbatch
        y_batch=[]#教師データのbatch
        
        
        #メモリからbatch_sizeだけデータを持ってくる
        #memory=[state,action,reward,next_state,terminal]
        batch_data=random.sample(self.replay_memory,BATCH_SIZE)
        
        for data in batch_data:
            #shape=(batch_size,4,:,:)
            state_batch.append(data[0])
            action_batch.append(data[1])
            reward_batch.append(data[2])
            next_state_batch.append(data[3])
            terminal_batch.append(data[4])
            
            
        #terminal_batchを数値化
        terminal_batch=[1 if batch==True else 0 for batch in terminal_batch ]
        
        #target_q_networkのvalue
        target_q_values_batch=self.target_q_values.eval(feed_dict={self.st:np.float32(np.array(next_state_batch)/255)})
        
        ##DDQN
        actions=np.argmax(self.main_q_values.eval(feed_dict={self.s:np.float32(np.array(next_state_batch)/255)}),axis=1)
        
        #推定した行動からtarget_q_valuesを変化させる
        target_q_values_batch=np.array([target_q_values_batch[i][action] for i,action in enumerate(actions)])
        
        #教師信号のバッチ
        y_batch=reward_batch+(1-terminal_batch)*GAMMA*target_q_values_batch
        
        #学習
        loss,_=self.sess.run([self.loss,self.optimizer],feed_dict={self.s:np.float32(np.array(state_batch)/255),self.a:action_batch,self.y:y_batch})
        
        self.total_loss+=loss
        
        
    def load_network(self):
        checkpoint=tf.train.get_checkpoint_state(SAVE_NETWORK_PATH)
        if checkpoint and checkpoint.model_checkpoint_path:
            self.saver.restore(self.sess,checkpoint.model_checkpoint_path)
            print('success load')
        else:
            print('training new network')
            
            
    
            
            
        
                
        
        
        
        

In [9]:
#データの前処理
def preprocessing(observation,before_observation):
    preprocessed_observation=np.maximum(observation,before_observation)
    preprocessed_observation=cv2.cvtColor(preprocessed_observation,cv2.COLOR_RGB2GRAY)
    preprocessed_observation=cv2.resize(preprocessed_observation,(FRAME_WIDTH,FRAME_HEIGHT))
    preprocesed_observation=np.uint8(preprocessed_observation/255)
    return np.reshape(preprocessed_observation,(1,FRAME_WIDTH,FRAME_HEIGHT))

In [ ]:
class Environment():
    
    def __init__(self):
        self.env=gym.make(ENV_NAME)
        #環境を定義したときに行動の個数と状態の個数は把握しておく必要性あり
        num_actions=self.env.action_space.n
        self.agent=Agent(num_actions)
        
    def run(self):
        if TRAIN:
            for _ in range(MAX_EPISODES):
                terminal=False
                observation=self.env.reset()
                
                #ここでランダムに初期化をしておく
                for _ in range(random.randint(0,NO_OP_STEPS=30)):
                    before_observation=observation
                    observation,_,_,_=self.env.step(0)#0は行動を起こさないということ
                state=preprocessing(observation,before_observation)
                self,agent.start=time.time()
                
                while not terminal:
                    before_observation=observation
                    action=self.agent.get_action(state)
                    observation,reward,terminal,_=self.env.step(action)
                    self.env.render()
                    processed_observation=preprocessing(observation,before_observation)
                    state=self.agent.run(state,action,reward,processed_observation,terminal)
                    
        #test mode        

In [ ]:
game_env=Environment()
game_env.run()